In [1]:
param_date = '20240724'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel(f'./{param_date}/1.group.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Group No'] = input_['Group No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Group'].tolist()

总数量：26


['Apparel & Gifts',
 'Belt Drive',
 'Body & Lamp Assembly',
 'Brake & Wheel Hub',
 'Cooling System',
 'Drivetrain',
 'Electrical',
 'Electrical-Bulb & Socket',
 'Electrical-Connector',
 'Electrical-Switch & Relay',
 'Engine',
 'Exhaust & Emission',
 'Fuel & Air',
 'Garage Equipment',
 'Hardware',
 'Heat & Air Conditioning',
 'Hoses/Lines & Clamps',
 'Ignition',
 'Interior',
 'Literature',
 'Steering',
 'Suspension',
 'Transmission-Automatic',
 'Transmission-Manual',
 'Wheel',
 'Wiper & Washer']

In [4]:
a = 0

a

0

In [5]:
crawler_status = 'error'

crawler_status

'error'

In [6]:
import requests
import json

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

payload = {'jsn': {'tab': 'extras',
                   'groupname': input_.loc[a, 'Group'],
                   'nodetype': 'groupname'}}

data = {'func': 'navnode_fetch',
        'payload': json.dumps(payload),
        'api_json_request': '1'}

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                 data=data,
                                 headers=get_header(),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        break

resp

1


<Response [200]>

In [7]:
json_ = resp.json()

json_

{'html_fill_sections': {'navchildren[]': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;extras&quot;,&quot;idepth&quot;:1,&quot;groupname&quot;:&quot;Apparel &amp; Gifts&quot;,&quot;subgroupname&quot;:&quot;RockAuto Apparel&quot;,&quot;nodetype&quot;:&quot;subgroupname&quot;}" /><input type="hidden" autocomplete="off" name="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" id="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nvlineicon"></td><td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]"><a href="/en/tools/apparel+&amp;+gifts,rockauto+apparel" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1__&quot;); return false;" tabindex="-1"><img src="

In [8]:
'redirect_to_url' in json_

False

In [9]:
json_['jsn']['tab'] == 'extras' and json_['jsn']['groupname'] == input_.loc[a, 'Group'] and json_['jsn']['nodetype'] == 'groupname' and 'collected_javascript' in json_

True

In [10]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
html = etree.HTML(str(soup))

with open('./year.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"extras","idepth":1,"groupname":"Apparel &amp; Gifts","subgroupname":"RockAuto Apparel","nodetype":"subgroupname"}'/>
   <input autocomplete="off" id="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" name="navnodeunique[extras-apparel &amp; gifts-rockauto apparel]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nvlineicon">
       </td>
       <td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]">
        <a href="/en/tools/apparel+&amp;+gifts,rockauto+apparel" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
        </a>
       </td>
      

In [11]:
list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

list_json

['{"groupindex":"__GIP__1__","tab":"extras","idepth":1,"groupname":"Apparel & Gifts","subgroupname":"RockAuto Apparel","nodetype":"subgroupname"}',
 '{"groupindex":"__GIP__2__","tab":"extras","idepth":1,"groupname":"Apparel & Gifts","subgroupname":"RockAuto Gift","nodetype":"subgroupname"}']

In [12]:
list_subgroup = [json.loads(json_)['subgroupname'] for json_ in list_json]

list_subgroup

['RockAuto Apparel', 'RockAuto Gift']

In [13]:
len(list_subgroup) == 0

False

In [14]:
df_temp = pd.DataFrame({'Group No': input_.loc[a, 'Group No'],
                        'Group': input_.loc[a, 'Group'],
                        'Subgroup No': [i+1 for i in range(len(list_subgroup))],
                        'Subgroup': list_subgroup})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Group No,Group,Subgroup No,Subgroup
0,1,Apparel & Gifts,1,RockAuto Apparel
1,1,Apparel & Gifts,2,RockAuto Gift


In [15]:
crawler_status = 'ok'

crawler_status

'ok'

In [16]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                             'Group': input_.loc[a, 'Group']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [17]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Group No', 'Subgroup No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_subgroup.xlsx', index=False)

output_correct

,Group No,Group,Subgroup No,Subgroup
0,1,Apparel & Gifts,1,RockAuto Apparel
1,1,Apparel & Gifts,2,RockAuto Gift


In [18]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Group No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_subgroup_error.xlsx', index=False)

output_error

""
